(1) You must decide (and explain in your report) how to specify the two input sequences. The sequence can e.g. be specified on the command-line, or they can be listed in an input file. The first choice is ok for short input sequences, while the second choice is more useful for longer input sequences. Feel free to implement both options. If you decide that the input sequences should be listed in an input file, then it might be convenient to use FASTA format, which is a widely used text based format for specifying sequences. There are many libraries for reading sequences in FASTA format depending on your programming language. Feel free to use any library you can find.


Mandatory: Make a program global_linear that implements global alignment using linear gap cost. The program should be implemented such that it takes at most quadratic time and space to compute the cost of an optimal global alignment. If requested, the program should output an optimal global alignment.



In [ ]:
import numpy as np


def backtrack(v, w, substitution_matrix, gapcost , alignment_matrix):
    n, m = len(v), len(w)
    v = '-' + v
    w = '-' + w
    backtrack_matrix = np.zeros((n+1, m+1), dtype=int)
    for i in range(1, n+1):
        for j in range(1, m+1):
            match = alignment_matrix[i-1, j-1] + substitution_matrix[v[i]][w[j]]
            delete = alignment_matrix[i-1, j] + gapcost
            insert = alignment_matrix[i, j-1] + gapcost
            alignment_matrix[i, j] = max(match, delete, insert)
            if alignment_matrix[i, j] == match:
                backtrack_matrix[i, j] = 0
            elif alignment_matrix[i, j] == delete:
                backtrack_matrix[i, j] = 1
            else:
                backtrack_matrix[i, j] = 2
    i, j = n, m
    v_aligned, w_aligned = '', ''
    while i > 0 and j > 0:
        if backtrack_matrix[i, j] == 0:
            v_aligned = v[i] + v_aligned
            w_aligned = w[j] + w_aligned
            i -= 1
            j -= 1
        elif backtrack_matrix[i, j] == 1:
            v_aligned = v[i] + v_aligned
            w_aligned = '-' + w_aligned
            i -= 1
        else:
            v_aligned = '-' + v_aligned
            w_aligned = w[j] + w_aligned
            j -= 1
    while i > 0:
        v_aligned = v[i] + v_aligned
        w_aligned = '-' + w_aligned
        i -= 1
    while j > 0:
        v_aligned = '-' + v_aligned
        w_aligned = w[j] + w_aligned
        j -= 1
    return v_aligned, w_aligned

def global_gap_cost_matrix(seq1:str, seq2:str, gap_cost:int, substitution_matrix:dict, alignment:bool=False):
    """ This function takes two sequences, a gap cost and a substitution matrix 
        and returns the cost matrix for the alignment of the two sequences. 
    """
    n = len(seq1) + 1
    m = len(seq2) + 1
    # Initialize the matrices
    M = np.zeros((n, m))
    for i in range(1, n):
        M[i][0] =  gap_cost * i
    for j in range(1, m):
        M[0][j] =  gap_cost * j
    for i in range(1, n):
        for j in range(1, m):
            substitution_cost = substitution_matrix[seq1[i-1]][seq2[j-1]]
            M[i][j] = max(M[i-1][j-1] + substitution_cost, M[i][j-1] + gap_cost, M[i-1][j] + gap_cost)
    if not alignment:
        return M[n-1][m-1]
    else:
        return backtrack(seq1, seq2, substitution_matrix, gap_cost, M)





Mandatory: Make a program global_affine that implements global alignment using affine gap cost. The program should be implemented such that it takes at most quadratic time and space to compute the cost of an optimal global alignment. If requested, the program should output an optimal global alignment.

In [ ]:
def affine_gap_cost_matrix(seq1, seq2, substitution_matrix, start_penalty, extention_penalty, alignment=False):
    n = len(seq1) + 1
    m = len(seq2) + 1

    # Initialize the matrices
    M = np.zeros((n, m))
    D = np.zeros((n, m))
    I = np.zeros((n, m))

    # Fill in the first row and column
    for i in range(1, n):
        M[i][0] = start_penalty + (i - 1) * extention_penalty
        I[i][0] = start_penalty + (i - 1) * extention_penalty
    for j in range(1, m):
        M[0][j] = start_penalty + (j - 1) * extention_penalty
        D[0][j] = start_penalty + (j - 1) * extention_penalty

    # Fill in the rest of the matrices
    for i in range(1, n):
        for j in range(1, m):
            substitution_cost = substitution_matrix[seq1[i-1], seq2[j-1]]
            M[i][j] = max(M[i-1][j-1] + substitution_cost, D[i-1][j-1] + substitution_cost, I[i-1][j-1] + substitution_cost)
            D[i][j] = max(M[i][j-1] + start_penalty, D[i][j-1] + extention_penalty)
            I[i][j] = max(M[i-1][j] + start_penalty, I[i-1][j] + extention_penalty)
    if not alignment:
        return M

(2) You must decide (and explain in your report) how to specify the character score matrix, and the parameters of the gap cost function, but aim for something simple. E.g. the parameters of the gap cost functions are specified as command line parameters, while the alphabet and character score matrix are specified in a control file read by the program. The file could e.g. be in the following "Phylip-like" format which is convenient for specifying both the score matrix and the alphabet. Here is an example of a score matrix for DNA in this format:

  4 
  A  0  5  2  5
  C  5  0  5  2
  G  2  5  0  5
  T  5  2  5  0
The first line in the file contains the number n of characters covered by the matrix. The next n lines describes the symbol for each character and its distance to the other characters. The above matrix says that matches (AA, CC, GG, TT) have distance 0, transitions (AG, GA, TC, CT) have distance 5, and transversions (AT, TA, AC, CA, GT, TG, GC, CG) have distance 2. If your input sequences contain characters which are not part of the specified alphabeth, your program should choose an appropriate action, e.g. terminate with an error message.


(3) The computed optimal alignment should be output in FASTA alignment format (also called Pearson format after the creator of the FASTA alignment program). In FASTA alignment format, the two aligned sequences are printed above each other with gaps inserted as described by the computed alignment. For example,

  >seq1 
  ctacgaaaggtcgtgtcacg-atgtcc------gc
  aagggatggcat---tgcatagaggaattgat--t
  gcaac

  >seq2
  ctt-----------------aatgtcccgcgta-c
  aagggatagcatgtg-gcatagaggaatagaata-
  gcagc
The FASTA alignment format is just one of many used alignment formats. The Jalview program makes it easy to read and display alignments in many different formats. Jalview can be used as an applet or downloaded and installed locally (see their www page for documentation and details).


Problem II: Experimenting with alignment methods
In the second part of this project you should design and implement experiments to examine the compuational qualities of your alignment methods. You should:

Construct test data and perform test runs to ensure that each of the programs perform as expected. In the report, you must explain how you have constructed test data to ensure that each program is tested properly. 

You should also verify that your global alignment performs as expected on the 4 test cases presented in project2_examples.txt.

Perform experiments that illustrate that the time consumption of the programs are as expected according to the theoretical bounds. E.g. by measuring the running time of your programs using the Unix time-command for sequences of increasing lengths.

Perform experiments that compare the running time for computing an optimal alignment using linear gap cost in quadratic space and the running time for computing an optimal alignment using affine gap cost. What do you expect, and do your experiments confirm this?

If you have implemented the optional parts above, you should also perform experiments that examine their running times.